In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [3]:
agent_data = pd.read_csv('Data/datastorm_agent_data.csv')
policy_data = pd.read_csv("Data/datastorm_policy_data_short.csv")

In [4]:
policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [6]:
snapshot_months = np.sort(pd.unique(policy_data["policy_snapshot_as_on"]))

# def divide_chunks(l, n):
#     # Divide months into k bins
#     for i in range(0, len(l), n): 
#         yield l[i:i + n]

# k = 6
# snapshot_months = list(divide_chunks(snapshot_months, k))

snapshot_months

array([20181231, 20190131, 20190228, 20190331, 20190430, 20190531,
       20190630, 20190731, 20190831, 20190901, 20191001, 20191101,
       20191201, 20200101, 20200201, 20200301, 20200401, 20200501,
       20200601, 20200701], dtype=int64)

In [50]:
valid_policy_data = policy_data[policy_data["termination_dt"].isnull()]
valid_policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [57]:
# Useful Columns
columns = ["main_holder_occupation_cd", "policy_term", "policy_payment_mode", "policy_status", "main_holder_gender", "main_holder_entry_age", "policy_snapshot_as_on" ,"main_holder_smoker_flag", "total_sum_assuared", "payment_method", "agent_code" ]

In [56]:
# Return cross sell status for each month
def is_cross_sell(no_of_policies):
    cross_sell = [];

    for k in range(len(no_of_policies)):
        n = 6
        if(k + n) >= len(no_of_policies) :
            n = len(no_of_policies) - k -1
        if(no_of_policies[k + n] > no_of_policies[k]):
            cross_sell.append(True)
        else:
            cross_sell.append(False)
    return cross_sell

new_data_set = pd.DataFrame(columns= columns + ["cross_sell"])

for client_name, client_data in valid_policy_data.groupby(["client_code"]):
    no_of_policies = client_data["policy_snapshot_as_on"].value_counts().sort_index()

    # Get only one data row for each month
    df = client_data.drop_duplicates("policy_snapshot_as_on", keep='last')
    df = df.filter(columns)
    df["cross_sell"] = is_cross_sell(list(no_of_policies))

    new_data_set = new_data_set.append(df)


    client_data
new_data_set

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,main_holder_gender,main_holder_entry_age,policy_snapshot_as_on,cross_sell
0,226.0,12,M,INFORCE,M,28,20190131,False
1,226.0,12,M,LAPSED,M,28,20200301,False
2,226.0,12,M,INFORCE,M,28,20181231,False
3,226.0,12,M,LAPSED,M,28,20191001,False
4,226.0,12,M,INFORCE,M,28,20190228,False
...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,F,40,20181231,False
9832,200.0,30,Q,INFORCE,F,40,20200201,False
9833,200.0,30,Q,INFORCE,F,40,20191001,False
9834,200.0,30,Q,INFORCE,F,40,20200101,False


In [ ]:
# categorical data
cat_data = []

drop_data = []

for element in drop_data:
    if element in cat_data:
        cat_data.remove(element)

def format_data(data_set, is_test=False):
    f_data_set = pd.get_dummies(data_set, prefix=cat_data, columns=cat_data)
    f_data_set = f_data_set.drop(drop_data, axis = 1)
    if(is_test):
        x = f_data_set
        global columns
        columns = x.columns
        return np.array(x)
    else:
        x = f_data_set.drop('cross_sell', axis = 1)
        y = f_data_set['cross_sell']
        return np.array(x), np.array(y)

x_train, y_train = format_data(train_data)
x_val, y_val = format_data(validation_data)
x_test = format_data(test_data, is_test=True)

print(len(x_train), len(x_val), len(x_test))
pd.DataFrame(y_val).head(10)

In [ ]:
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

pd.DataFrame(x_train, columns=columns).head(10)

In [ ]:

classifier = XGBClassifier(n_estimators=15, max_depth=5)

classifier.fit(x_train_res, y_train_res)

y_pred = classifier.predict(x_val)
y_pred2 = classifier.predict(x_val2)

print('Accuracy =', classifier.score(x_val, y_val))
print('F1 Score =', roc_auc_score(y_val, y_pred, average='macro'))

print('Accuracy =', classifier.score(x_val2, y_val2))
print('F1 Score =', roc_auc_score(y_val2, y_pred2, average='macro'))

In [199]:
# Nullify Terminated Users

terminated_users = policy_data[policy_data["termination_dt"].notna()].client_code.unique()

In [203]:
test_data = pd.read_csv("Data/testset.csv")

# -- THIS IS NOT FINAL --
test_data["probability_of_cross_sell"] = np.random.rand(test_data.shape[0])

test_data.loc[test_data["map_client_cd"].isin(terminated_users) , "probability_of_cross_sell"] = 0

test_data


,map_client_cd,probability_of_cross_sell
0,C100003,0.406901
1,C100004,0.989584
2,C100009,0.339157
3,C100014,0.301586
4,C100015,0.536124
...,...,...
19137,C154448,0.549853
19138,C154449,0.740576
19139,C154450,0.841277
19140,C154454,0.732790


In [204]:
test_data.to_csv("Data/submission.csv")
